# Crear el dataset sobre personoas pensionadas en méxico

## Vamos a importar las librerias que sean necesarias para crear el conjnto de datos del ISSTE

In [2]:
import requests
import pandas as pd
from io import StringIO
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.max_columns', None)

## Definimos la función para obtener los datos de la página (opcional descargalos)

In [ ]:
def bajar_datos():  
    url = "https://www.issste.gob.mx/datosabiertos/pensionadosdirectos.txt"

    response = requests.get(url)
    response.encoding = 'utf-8'

    data = StringIO(response.text)
    df = pd.read_csv(data, sep="\t", dtype=str) 
    
    #df.to_csv('ISSSTE_data.csv', index=False)  #Opcional: guardar el archivo CSV localmente
    return df

## Función para limpiar los datos

In [17]:
def limpiar_datos(df):
    df.drop(columns=['anio','mes','primer_ramo', 'primer_ramo_des', 'ult_ramo','ult_ramo_des','nombre','paterno', 'materno','ent_pen','alta_nomina','importe_pen',
                'sexo','tiempo_serv','ent_pen_des','ent_pago','tip_pen_act','tip_pen_org','tip_pen_org_des','porcentaje','tip_pen_act','tip_pen_act_des','importe_msm'], inplace=True)
    
    df.dropna(inplace=True)
    
    df = df[df['primera_fecha'] >= '1959-12-30']
    df = df[df['primera_fecha'] < df['ult_fecha']]

    df['primera_fecha'] = pd.to_datetime(df['primera_fecha'])
    df['ult_fecha'] = pd.to_datetime(df['ult_fecha'])
    df['fecha_inidi'] = pd.to_datetime(df['fecha_inidi'])

    df['ent_pago_des'] = df['ent_pago_des'].str.replace('DELEGACION ESTATAL EN ','')
    df['ent_pago_des'] = df['ent_pago_des'].str.replace('DELEGACION EN LA ZONA ','')
    df['ent_pago_des'] = df['ent_pago_des'].str.replace('DEL D.F.','DE LA CDMX')

    antes = df[df['primera_fecha'] < '1997-07-01']
    antes.reset_index(drop=True, inplace=True)
    despues = df[df['primera_fecha'] >= '1997-07-01']
    despues.reset_index(drop=True, inplace=True)

    return antes, despues   

## La función principal

In [ ]:
if __name__ == "__main__":
    df = bajar_datos()
    #df = pd.read_csv('ISSSTE_data.csv')   # Si lo tienes guardado localmente
    antes, despues = limpiar_datos(df)
    antes.to_csv('antes_del_97.csv', index=False)
    despues.to_csv('despues_del_97.csv', index=False)